In [7]:
import pandas as pd
#ip = individual pitcher
PATH_IP = "C:/Users/home/DA(inflearn)/individual_pitcher/2020빅콘테스트 문제데이터(데이터분석분야-퓨처스리그)/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_개인투수_"
PATH_TP = "C:/Users/home/DA(inflearn)/individual_pitcher/2020빅콘테스트 문제데이터(데이터분석분야-퓨처스리그)/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_"
season = [2016, 2017, 2018, 2019, 2020]

# 개인

In [20]:
def load_csv(path_ip, path_tp, season):
    doc_ip = None
    doc_tp = None
    for i in season:
        doc_i = pd.read_csv(path_ip+str(i)+".csv")
        doc_ip = pd.concat([doc_ip,doc_i], axis=0)
        doc_j = pd.read_csv(path_tp+str(i)+".csv")
        doc_tp = pd.concat([doc_tp,doc_j], axis=0)
    doc = pd.merge(doc_ip,doc_tp[["G_ID","T_ID","VS_T_ID","WLS"]],on=["G_ID","T_ID","VS_T_ID"], how="left")
    doc = doc.set_index(pd.Index(range(0,doc.shape[0])))
    
    for i, row in doc.iterrows():
        if doc.at[i,"WLS_y"]=="W":
            doc.at[i,"WLS_y"] = 1
        else:
            doc.at[i,"WLS_y"] = 0
    doc[["WLS_y"]] = doc[["WLS_y"]].astype(str).astype("int64")
    return doc
df = load_csv(PATH_IP, PATH_TP, season)
df[["WLS_y"]].dtypes

WLS_y    int64
dtype: object

# CYP: ( 이닝/2 - 자책점 ) + 삼진/10 + 승리

In [22]:
def convert_CYP(df):
    needs = df.copy()
    
    for i, row in needs.iterrows():
        needs.at[i,"G_ID"] = needs.at[i,"G_ID"][:4]
    needs = needs.groupby(by=["G_ID","P_ID"]).sum()
    needs = needs[["INN2","ER","KK","WLS_y"]]
    needs["INN2"] = needs["INN2"]/3
    cyp = ((needs["INN2"]/2)-needs["ER"])+needs["KK"]/10+needs["WLS_y"]
    return cyp
cyp = convert_CYP(df)
cyp

G_ID  P_ID 
2016  60105    -1.066667
      60146    15.400000
      60164     0.666667
      60181    37.100000
      60263    27.233333
                 ...    
2020  79545     1.700000
      79764    -8.866667
      79825     3.200000
      79847    -2.933333
      99445    -0.933333
Length: 1240, dtype: float64

# 개인

In [26]:
def load_csv(path_tp, season):
    doc_tp = None
    for i in season:
        doc_i = pd.read_csv(path_tp+str(i)+".csv")
        doc_tp = pd.concat([doc_tp,doc_i], axis=0)
    doc = doc_tp.set_index(pd.Index(range(0,doc_tp.shape[0])))
    
    for i, row in doc.iterrows():
        if doc.at[i,"WLS"]=="W":
            doc.at[i,"WLS"] = 1
        else:
            doc.at[i,"WLS"] = 0
    doc[["WLS"]] = doc[["WLS"]].astype(str).astype("int64")
    return doc
df = load_csv(PATH_TP, season)
df

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,CG_CK,WLS,HOLD,INN2,...,KK,GD,WP,BK,ERR,R,ER,P_WHIP_RT,P2_WHIP_RT,CB_WHIP_RT
0,20160401HHLG0,20160401,LG,HH,0,B,0,1,0,36,...,10,1,0,0,0,4,4,0.642857,1.285714,2.400000
1,20160401HHLG0,20160401,HH,LG,0,T,0,0,0,34,...,11,0,1,0,0,5,4,1.500000,1.000000,0.750000
2,20160401HTNC0,20160401,NC,HT,0,B,0,1,0,27,...,10,1,2,0,0,4,4,1.333333,1.038462,2.142857
3,20160401HTNC0,20160401,HT,NC,0,T,0,0,0,24,...,9,1,0,0,0,5,5,0.500000,1.695652,1.875000
4,20160401KTSK0,20160401,SK,KT,0,B,0,0,0,27,...,7,0,0,0,0,8,8,1.000000,2.357143,2.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,20200719,LT,SS,0,T,0,1,2,27,...,9,1,0,0,0,1,1,0.600000,1.111111,1.500000
6396,20200719OBHT0,20200719,HT,OB,0,B,0,0,0,27,...,4,0,0,0,0,8,8,2.142857,1.800000,0.750000
6397,20200719OBHT0,20200719,OB,HT,0,T,0,1,1,27,...,5,1,0,0,0,4,4,3.000000,1.000000,1.875000
6398,20200719WOSK0,20200719,SK,WO,0,B,0,1,0,27,...,4,1,0,0,0,3,3,1.500000,1.105263,2.142857


In [28]:
def convert_team_CYP(df):
    needs = df.copy()
    
    for i, row in needs.iterrows():
        needs.at[i,"G_ID"] = needs.at[i,"G_ID"][:4]
    needs = needs.groupby(by=["G_ID","T_ID"]).sum()
    needs = needs[["INN2","ER","KK","WLS"]]
    needs["INN2"] = needs["INN2"]/3
    cyp = ((needs["INN2"]/2)-needs["ER"])+needs["KK"]/10+needs["WLS"]
    return cyp
team_cyp = convert_team_CYP(df)
team_cyp

G_ID  T_ID
2016  HH      -17.066667
      HT       99.500000
      KT      -50.500000
      LG       84.733333
      LT        8.066667
      NC      200.866667
      OB      199.033333
      SK      111.500000
      SS       -4.333333
      WO      101.666667
2017  HH       41.900000
      HT      143.500000
      KT      -33.700000
      LG      201.500000
      LT      181.266667
      NC      158.033333
      OB      196.600000
      SK       95.333333
      SS      -35.033333
      WO       95.000000
2018  HH      128.400000
      HT       44.033333
      KT       42.666667
      LG       62.133333
      LT       60.233333
      NC       22.000000
      OB      126.833333
      SK      166.333333
      SS       81.066667
      WO       95.100000
2019  HH      108.733333
      HT      138.600000
      KT      186.300000
      LG      264.966667
      LT      104.800000
      NC      242.266667
      OB      315.833333
      SK      343.366667
      SS      132.066667
      WO      

In [119]:
team_pitcher = pd.read_csv("C:/Users/home/DA(inflearn)/individual_pitcher/팀투수_세이버 매트릭스_합본.csv")
team_pitcher

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,...,홈런율,삼진율,K9,BB9,HR9,KBB,P/G,P/IP,P/PA,PFR
0,0,0,0,0,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,1,1,20160401HHLG0,2016-04-01,HH,LG,0,T,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,2,2,2,20160401HTNC0,2016-04-01,NC,HT,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,3,3,3,20160401HTNC0,2016-04-01,HT,NC,0,T,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,4,4,4,20160401KTSK0,2016-04-01,SK,KT,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,6395,6395,6395,635,20200719LTSS0,2020-07-19,LT,SS,0,T,...,0.030008,0.174134,6.774665,3.091352,1.167479,2.191489,149.557377,16.668088,3.842881,1.110840
6396,6396,6396,6396,636,20200719OBHT0,2020-07-19,HT,OB,0,B,...,0.020524,0.199127,7.675810,3.114090,0.791147,2.464865,145.836066,16.638404,3.881326,1.202618
6397,6397,6397,6397,637,20200719OBHT0,2020-07-19,OB,HT,0,T,...,0.019154,0.166800,6.697923,3.188724,0.769139,2.100503,154.793651,17.362611,3.879077,1.112760
6398,6398,6398,6398,638,20200719WOSK0,2020-07-19,SK,WO,0,B,...,0.023089,0.176354,7.102732,4.248812,0.929929,1.671698,154.640625,17.631235,3.933625,1.268409


In [120]:
team_pitcher.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       '게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '평균자책점', 'RA/9', 'ERC', 'FIP', 'LOB',
       '홈런율', '삼진율', 'K9', 'BB9', 'HR9', 'KBB', 'P/G', 'P/IP', 'P/PA', 'PFR'],
      dtype='object')

In [121]:
pitcher = pd.read_csv("C:/Users/home/DA(inflearn)/individual_pitcher/개인투수_세이버 매트릭스_합본.csv")
pitcher

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,...,홈런율,삼진율,K9,BB9,HR9,KBB,P/G,P/IP,P/PA,PFR
0,0,0,0,0,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,1,1,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,2,2,2,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,3,3,3,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,4,4,4,20160401HHLG0,2016-04-01,HH,LG,0,T,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27799,27799,27799,27799,2882,20200719WOSK0,2020-07-19,WO,SK,0,T,...,0.030928,0.185567,7.043478,3.130435,1.173913,2.250000,146.261538,16.782609,3.938776,1.173913
27800,27800,27800,27800,2883,20200719WOSK0,2020-07-19,WO,SK,0,T,...,0.019608,0.205882,7.985915,3.422535,0.760563,2.333333,146.261538,17.197183,3.990196,1.267606
27801,27801,27801,27801,2884,20200719WOSK0,2020-07-19,WO,SK,0,T,...,0.027027,0.270270,9.642857,4.821429,0.964286,2.000000,146.261538,14.785714,3.729730,1.607143
27802,27802,27802,27802,2885,20200719WOSK0,2020-07-19,WO,SK,0,T,...,0.000000,0.075472,3.724138,3.724138,0.000000,1.000000,146.261538,18.103448,3.301887,0.827586


In [122]:
pitcher.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       '게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '선수코드', '선발', '구원', '완투',
       '종료', '결과', '홀드', '이닝*3', '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런',
       '도루', '도루.1', '희타', '희비', '4구', '고4', '사구', '삼진', '병살', '폭투', '보크',
       '실책', '실점', '자책점', '득점권WHIP', '2점차이하WHIP', '345번타자WHIP', '시즌', '평균자책점',
       'RA/9', 'ERC', 'FIP', 'LOB', '홈런율', '삼진율', 'K9', 'BB9', 'HR9', 'KBB',
       'P/G', 'P/IP', 'P/PA', 'PFR'],
      dtype='object')

In [123]:
wsl = team_pitcher["결과"]
perWSL = pitcher["결과"]
def win_lose(row):
    if row["결과"] == "W":
        row["결과"] = 1
    else:
        row["결과"] = 0
    return row
team_pitcher = team_pitcher.apply(win_lose,axis=1)
team_pitcher["결과"]

0       1
1       0
2       1
3       0
4       0
       ..
6395    1
6396    0
6397    1
6398    1
6399    0
Name: 결과, Length: 6400, dtype: int64

In [124]:
pitcher = pitcher.apply(win_lose,axis=1)
pitcher["결과"]

0        1
1        0
2        0
3        0
4        0
        ..
27799    0
27800    0
27801    0
27802    0
27803    0
Name: 결과, Length: 27804, dtype: int64

In [125]:
#( 이닝/2 - 자책점 ) + 삼진/10 + 승리
def cyp(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['CYP'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = (previous_sum["이닝*3"]/6-previous_sum["자책점"])+previous_sum["삼진"]/10+previous_sum["결과"]
    except ZeroDivisionError:
        row['CYP'] = 0
    row["CYP"] = pg
    return row
def perCYP(row):
    previous = pitcher[pitcher['선수코드'] == row['선수코드']].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    previous_sum = previous.sum()
    if len(previous) == 0:
        row['CYP'] = 0
        return row
    try:
        pg = (previous_sum["이닝*3"]/6-previous_sum["자책점"])+previous_sum["삼진"]/10+previous_sum["결과"]
    except ZeroDivisionError:
        row['CYP'] = 0
        return row
    row['CYP'] = pg
    return row

In [132]:
team_pitcher = team_pitcher.apply(cyp,axis=1)
team_pitcher["결과"] = wsl
team_pitcher["CYP"].tail()

6395    50.866667
6396    90.933333
6397    47.633333
6398    40.966667
6399    62.500000
Name: CYP, dtype: float64

In [106]:
team_pitcher["결과"]

0       W
1       L
2       W
3       L
4       L
       ..
6395    W
6396    L
6397    W
6398    W
6399    L
Name: 결과, Length: 6400, dtype: object

In [126]:
pitcher = pitcher.apply(perCYP,axis=1)
pitcher["결과"] = perWSL
pitcher["CYP"]

0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
           ...   
27799    3.300000
27800    1.933333
27801    1.666667
27802   -6.766667
27803    0.700000
Name: CYP, Length: 27804, dtype: float64

In [134]:
team_pitcher.to_csv("팀투수_세이버 매트릭스_합본.csv",encoding='utf-8-sig')
pitcher.to_csv("개인투수_세이버 매트릭스_합본.csv",encoding='utf-8-sig')

In [133]:
team_pitcher

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,...,삼진율,K9,BB9,HR9,KBB,P/G,P/IP,P/PA,PFR,CYP
0,0,0,0,0,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,1,1,20160401HHLG0,2016-04-01,HH,LG,0,T,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,2,2,2,20160401HTNC0,2016-04-01,NC,HT,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,3,3,3,20160401HTNC0,2016-04-01,HT,NC,0,T,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,4,4,4,20160401KTSK0,2016-04-01,SK,KT,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,6395,6395,6395,635,20200719LTSS0,2020-07-19,LT,SS,0,T,...,0.174134,6.774665,3.091352,1.167479,2.191489,149.557377,16.668088,3.842881,1.110840,50.866667
6396,6396,6396,6396,636,20200719OBHT0,2020-07-19,HT,OB,0,B,...,0.199127,7.675810,3.114090,0.791147,2.464865,145.836066,16.638404,3.881326,1.202618,90.933333
6397,6397,6397,6397,637,20200719OBHT0,2020-07-19,OB,HT,0,T,...,0.166800,6.697923,3.188724,0.769139,2.100503,154.793651,17.362611,3.879077,1.112760,47.633333
6398,6398,6398,6398,638,20200719WOSK0,2020-07-19,SK,WO,0,B,...,0.176354,7.102732,4.248812,0.929929,1.671698,154.640625,17.631235,3.933625,1.268409,40.966667


In [130]:
pitcher

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,...,삼진율,K9,BB9,HR9,KBB,P/G,P/IP,P/PA,PFR,CYP
0,0,0,0,0,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,1,1,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,2,2,2,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,3,3,3,20160401HHLG0,2016-04-01,LG,HH,0,B,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,4,4,4,20160401HHLG0,2016-04-01,HH,LG,0,T,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27799,27799,27799,27799,2882,20200719WOSK0,2020-07-19,WO,SK,0,T,...,0.185567,7.043478,3.130435,1.173913,2.250000,146.261538,16.782609,3.938776,1.173913,3.300000
27800,27800,27800,27800,2883,20200719WOSK0,2020-07-19,WO,SK,0,T,...,0.205882,7.985915,3.422535,0.760563,2.333333,146.261538,17.197183,3.990196,1.267606,1.933333
27801,27801,27801,27801,2884,20200719WOSK0,2020-07-19,WO,SK,0,T,...,0.270270,9.642857,4.821429,0.964286,2.000000,146.261538,14.785714,3.729730,1.607143,1.666667
27802,27802,27802,27802,2885,20200719WOSK0,2020-07-19,WO,SK,0,T,...,0.075472,3.724138,3.724138,0.000000,1.000000,146.261538,18.103448,3.301887,0.827586,-6.766667
